In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [6]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [10]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [15]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={'id':'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = 'team'
    cols[-1] = 'total'
    line_score.columns = cols
    
    line_score = line_score[['team', 'total']]
    return line_score

In [18]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={'id': f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [46]:
def read_season_info(soup):
    nav = soup.select('#bottom_nav_container')[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [53]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score['team'])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"


    full_game = pd.concat([game, game_opp], axis=1)

    full_game["seasons"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 8900
200 / 8900
300 / 8900
400 / 8900
500 / 8900
600 / 8900
700 / 8900
800 / 8900
900 / 8900
1000 / 8900
1100 / 8900
1200 / 8900
1300 / 8900
1400 / 8900
1500 / 8900
1600 / 8900
1700 / 8900
1800 / 8900
1900 / 8900
2000 / 8900
2100 / 8900
2200 / 8900
2300 / 8900
2400 / 8900
2500 / 8900
2600 / 8900
2700 / 8900
2800 / 8900
2900 / 8900
3000 / 8900
3100 / 8900
3200 / 8900
3300 / 8900
3400 / 8900
3500 / 8900
3600 / 8900
3700 / 8900
3800 / 8900
3900 / 8900
4000 / 8900
4100 / 8900
4200 / 8900
4300 / 8900
4400 / 8900
4500 / 8900
4600 / 8900
4700 / 8900
4800 / 8900
4900 / 8900
5000 / 8900
5100 / 8900
5200 / 8900
5300 / 8900
5400 / 8900
5500 / 8900
5600 / 8900
5700 / 8900
5800 / 8900
5900 / 8900
6000 / 8900
6100 / 8900
6200 / 8900
6300 / 8900
6400 / 8900
6500 / 8900
6600 / 8900
6700 / 8900
6800 / 8900
6900 / 8900
7000 / 8900
7100 / 8900
7200 / 8900
7300 / 8900
7400 / 8900
7500 / 8900
7600 / 8900
7700 / 8900
7800 / 8900
7900 / 8900
8000 / 8900
8100 / 8900
8200 / 8900
8300 / 8900
8400 / 8900
8

In [55]:
games_df = pd.concat(games, ignore_index=True)

In [57]:
games_df.to_csv("nba_games.csv")